# Text parser for volume two

In [45]:
# DO NOT USE

import os
with open("./input/raw.txt", "r") as infile:
    data = infile.readlines()
    
    # Usable Parts of Section Two
    with open("input/Southern.txt", 'w') as outfile:
        outfile.writelines(data[3074:18607])
        
    # Usable Parts of Section Three
    with open("Northern.txt", 'w') as outfile:
        outfile.writelines(data[18611:39874])
        
    # Usable Parts of Section Four
    with open("Jiuliyang.txt", 'w') as outfile:
        outfile.writelines(data[39727:43643])

In [32]:
# DO NOT USE

import re


def refine(file):
    # Function to process raw data for future further operation
    
    buffer = None
    
    with open(file, "r+") as f:
        buffer = f.read()
        f.seek(0)
        
        # Delete Watermarks
        buffer = re.sub("Kenneth Dean and Zhenman ZHENG - 978-90-47-44017-8", "", buffer).rstrip()
        buffer = re.sub("Downloaded from Brill.com02/18/2020 04:31:15AM", "", buffer).rstrip()
        buffer = re.sub("via National University of Singapore - YNC Library", "", buffer).rstrip()
        
        # Delete Headers
        buffer = re.sub("the southern irrigated plain 南洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the northern irrigated plain 北洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the jiuliyang irrigated plain 九里洋水利系统", "", buffer).rstrip()
        buffer = re.sub("part (one|two|three|four|five)", "", buffer).rstrip()
        
        # Delete Page Numbers
        buffer = re.sub("\n[0-9]+\n", "", buffer).rstrip()
        buffer = re.sub("\f(.*?)\n", "", buffer).rstrip()
        buffer = re.sub("\n\n(A|V)[0-9]+\n\n\n", "", buffer).rstrip()
        
        # Delete blank lines
        buffer = re.sub("\n+", " ", buffer).rstrip()
        
        # Add line breaks at each new entry
        buffer = re.sub("((A|V)[0-9]{3})", "\n\g<1>", buffer)

        f.truncate()        
        f.write(buffer)

In [46]:
# DO NOT USE

refine("Southern.txt")
refine("Northern.txt")
refine("Jiuliyang.txt")



In [1]:
# Read god names and create JSON of them
import os, json

godsList= []

with open("input/list_of_gods.txt", 'r') as infile:
    data = infile.readlines()
    
    for line in data:
        EngText=" ".join(line.split()[:-1]).split("/")
        
        lineEntry = {
            "Name": EngText[0],
            "Name_zh": line.split()[-1]
        }
        
        if len(EngText) > 1:
            lineEntry["Aliases"] = EngText[1:]
        else:
            lineEntry["Aliases"] = []
        
        godsList.append(lineEntry)
        
    infile.close()

with open("./output/god_names.json", "w") as outfile:
                json.dump(godsList, outfile, ensure_ascii=False)
                outfile.close()

In [2]:
# Load village mapping functions
import re, os, json

village_index = []

with open("./input/contents.txt") as infile:
    for line in infile:
        buffer = re.split("(V[0-9]{3})", line)      
        entry = {
            "UID_A": buffer[0][0:4],
            "Name": " ".join(buffer[0].split(" ")[1:-2]),
            "Name_zh": buffer[0].split(" ")[-2]
        }

        runs = int((len(buffer)-1)/2 + 1)
        villageList=[]

        for i in range(1,runs):
            village_entry = {
                "UID_V": buffer[i*2-1],
                "Name": buffer[i*2].split(" ")[1],
                "Name_zh": buffer[i*2].split(" ")[2]
            }
            villageList.append(village_entry)

        entry["Num_Villages"] = len(villageList)
        entry["Villages"] = villageList

        village_index.append(entry)

with open("./output/village_index.json", "w") as outfile:
    json.dump(village_index, outfile, ensure_ascii=False)
    outfile.close()

infile.close()
    
#search function
def whatAlliance(searchterm):
    for alliance in village_index:
        for village in alliance["Villages"]:
            if village["UID_V"] == searchterm:
                return alliance
        
    return -1
    

In [3]:
# Load surname mapping functions

with open('./input/Surname_Groups.json', "r") as surnameFile:
    surnameGroups = json.load(surnameFile)
surnameFile.close()

#search function
def whatSurnames(searchterm):
    for village in surnameGroups:
        if village["UID"] == searchterm:
            return village
    return -1


In [25]:
# Load Text and print them into seperate text files.
import json, regex

def createVillageFiles(path, irrNetwork):
    
    with open(path) as infile:
        for line in infile:
            if line[0] == "V":
                uid = line[0:4]                
                buffer1 = line.split(" ", 3)
                buffer2 = buffer1[3].partition("2. Surname groups")
                buffer3 = buffer2[2].partition("3. Village temples")
                buffer4 = buffer3[2].partition("4. Rituals")
                buffer5 = buffer4[2].partition("Birthday celebrations of the gods")
            
                # Search for Alliance details
                allianceDetails = whatAlliance(uid)
                
                temples = regex.split("(?:^|\.\s)\d+\)(?=\s[A-Z])", buffer4[0])
                print(temples)
            
                entry = {
                    "UID": buffer1[0],
                    "Name": buffer1[1],
                    "Name_zh": buffer1[2],
                    "Alliance": allianceDetails["Name"],
                    "Alliance_UID": allianceDetails["UID_A"],
                    "Alliance_zh": allianceDetails["Name_zh"],
                    "Irrigation_Region": irrNetwork,
                    "Raw_Text": {                        
                        "Village_Settlement": buffer2[0][22:],
                        "Surname_Groups": buffer3[0][1:],
                        "Village_Temples": buffer4[0][1:],
                        "Rituals": {
                            "Yuanxiao processions": buffer5[0][23:],
                            "Birthday celebrations of the gods": buffer5[2][1:].strip()
                        }
                    }
                }
                
                # Add administrative village if applicable
                admin_village = re.search("\w* Administrative Village\s([^\.\s]+)", entry["Raw_Text"]["Village_Settlement"])
                if admin_village != None:
                    split_words = admin_village.group(0).split(" ")
                    entry["Raw_Text"]["Administrative_Village"] = " ".join(split_words[:-1])
                    entry["Raw_Text"]["Administrative_Village_zh"] = "".join(split_words[-1])                    
                else:
                    entry["Administrative_Village"] = "-1"
                    entry["Administrative_Village_zh"] = "-1"
                    
               # Find Population
                pop = re.search("[0-9,]*(?=\speople|\sresidents|\sinhabitants)", entry["Raw_Text"]["Village_Settlement"])
                
                if pop == None:
                    pop = re.search("[\w\s]*(people|residents|inhabitants|population)[\w\s]*", entry["Raw_Text"]["Village_Settlement"])
                    
                    if pop == None:
                        entry["Population"] = -1
                    else:                    
                        pop = re.search("[0-9,]*", pop.group(0))
                    
                        if pop == None:
                            entry["Population"] = -1
                        else:
                            entry["Population"] = pop.group(0)
                else:
                    entry["Population"] = pop.group(0)
                
                # Number of temples
                # Main Gods Worshipped
                Main_Gods = []
                num_temple = 0
                for templedata in re.finditer("\.*(main god|Main altar|dedicated)([^.]+)\.", entry["Raw_Text"]["Village_Temples"], flags=re.IGNORECASE):
                    # print(uid, templedata)
                    for god in godsList:
                        if god["Name"] in templedata.group(0):
                            if not god["Name"] in Main_Gods:
                                Main_Gods.append(god["Name"])
                        else:
                            for aliases in god["Aliases"]:
                                if aliases in templedata.group(0):
                                    if not god["Name"] in Main_Gods:
                                        Main_Gods.append(god["Name"])
                                        
                    num_temple = num_temple + 1
                
                entry["Num_Temples"] = str(num_temple)
                entry["Main_Gods"] = Main_Gods
                entry["Num_Main_Gods"] = str(len(Main_Gods))
                
                # Secondary Gods Worshipped
                Sec_Gods = []
                for templedata in re.finditer("\.*(secondary)([^.]+)\.", entry["Raw_Text"]["Village_Temples"], flags=re.IGNORECASE):
                    # print(uid, templedata)
                    for god in godsList:
                        if god["Name"] in templedata.group(0):
                            if not god["Name"] in Sec_Gods:
                                Sec_Gods.append(god["Name"])
                        else:
                            for aliases in god["Aliases"]:
                                if aliases in templedata.group(0):
                                    if not god["Name"] in Sec_Gods:
                                        Sec_Gods.append(god["Name"])
                
                entry["Secondary_Gods"] = Sec_Gods
                entry["Num_Secondary_Gods"] = str(len(Sec_Gods))
                
                villages.append(entry) 
                
    infile.close()
    
    # dump to file
    with open("./output/village_master.json", "w") as outfile:
        json.dump(villages, outfile, ensure_ascii=False)
        outfile.close()

    print("completed parsing", path)
    return villages
              

In [26]:
villages = []

overview = createVillageFiles("./input/1-100.txt", "Southern")
overview = createVillageFiles("./input/101-200.txt", "Southern")
overview = createVillageFiles("./input/201-297.txt", "Southern")
overview = createVillageFiles("./input/298-400.txt", "Northern")
overview = createVillageFiles("./input/401-500.txt", "Northern")
overview = createVillageFiles("./input/501-600.txt", "Northern")
overview = createVillageFiles("./input/601-659.txt", "Northern")
overview = createVillageFiles("./input/jiuliyang.txt", "Jiuliyang")
print("Completed")

[' 1) Qianfeigong 钱妃宫 was originally called the Zhenhuimiao 贞惠庙, and later the Qianlinggong 钱灵宫. The temple was built during the Northern Song in Yuanfeng 7 (1083). The temple was renovated during the early Ming. In 1967 the temple was demolished, but it was rebuilt in 1985. The main god is Qianshi shengfei 钱氏圣妃. Secondary gods are Guangping Zhouwang 广平周王 and Taishan Kongwang 泰山孔王. The stone inscription composed in the Xianchun period (1265–1274) by Liu Kezhuang 刘克庄 entitled 协应钱夫人庙记 (Record of the Temple to Lady Qian of Beneficial Assistance) (Epigraphical Materials, 1995:54, No. 48) is about this temple (stele no longer extant)', ' Xinglongshê 兴隆社: The main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. ']
[' 1) Xindunshê 新墩社 was built in the late Qing. The main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人', ' Minganci 明安祠 is dedicated to Sanyi jiaozhu 三一教主. ']
[' 1) Mulan zushê 木兰祖社 is co-owned by Tiezao 铁灶. The main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Seco

[' 1) Xiajiangjing Baoshangong 霞江境宝山宫: This temple branched off from the Fushangong 福山宫; it was originally built during the Ming Jiajing period (1522–66) and rebuilt in Xianfeng 11 (1861). It was renovated in 1946 and in 1987. The main god is Ciji zhenjun 慈济 真君. Secondary gods include Taiyi xiangu 太乙仙姑, Wuhu jialuo 五虎 迦罗, Liugong yuanshuai 刘公元帅, Zhanggong shengjun 张公圣君, Nezha taizi 哪吒太子, and Ligong yuanshuai 李公元帅', ' Wenshengshê 文盛社: Main gods: Zunzhu mingwang 尊主明王, Houtu furen 后土夫人', ' Xiajiangjing Longxiantan 霞江境隆显坛 is the main temple of the village. Originally built in Qianlong 30 (1765), it was rebuilt twice: in Daoguang 22 (1842), and in 1932. It was later renovated in 1986. Middle altar: Yanggong taishi 杨公太师, Magong taishi 马公太师, Tiegong taishi 铁公太师, Zhanggong shengjun 张公圣君, Tiangong yuanshuai 田公元帅. Left altar: Tianshang shengmu 天上圣母. Right altar: Dutong shenghou 都统圣侯. Front: Fenghuo ertong 风火儿童, Baiya jiangjun 白牙将军', ' Baozhou zumiao 宝洲祖庙 is dedicated to Tianshang shengmu 天上 圣母',

[' 1) Linfeng shushê 麟峰书社: The main gods are Zunzhu mingwan 尊主明王 and Houtu furen 后土夫人', ' Xialin zhushê 下林朱社, Julongge 聚龙阁: The main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Secondary gods include Sima shengwang 司马圣王 and Fude zhengshen 福德正神', ' Julanting 聚澜亭: The main god is Guanyin dashi 观音大士. Secondary gods include Weituo 韦驮and Jialan 迦蓝. ']
[' 1) Xingshangong 兴山宫, Anping shushê 安平书社: The main gods are Zunzhu mingwang 尊主明王, Houtu furen 后土夫人, and Yuantian shangdi 元天上帝. Secondary gods include Tianshang shengmu 天上圣母 and Guanyin dashi 观音大士', ' Junzhentang 郡真堂: The main god is Lingguan dadi 灵官大帝. ']
[' 1) Hudong shushê 湖东书社: The main gods are Zunzhu mingwan 尊主明王, Houtu furen 后土夫人, and Tianshang shengmu 天上圣母. Secondary gods include Fude zhengshen 福德正神', ' Lingnan shuyuan 岭南书院: Also called Hulinggong 湖岭宫, the temple was built in Qianlong 34 (1769), and was renovated both in Tongzhi 12 (1873) and in 1947. The main god is Wenchang dijun 文昌帝君. Secondary gods include Zhanggong shengj

[' (the main temple for the alliance is the Haopushê 濠浦社 in Xuqiao 徐桥) 1) Xinglongmiao 兴隆庙: The main god is Xuantian shangdi 玄天上帝', ' Xinxingci 新兴祠: The main god is Sanyi jiaozhu 三一教主', ' Xinxingmiao 新兴庙: The main god is Xufe daren 徐府大人. Secondary gods include Qixiannü 七仙女 and Zhuo zhenren 卓真人. ']
[' Wan’anshê 万安社 was built during the late Qing, and renovated both during the Republican era and after the founding of PRC. The main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Secondary gods include Tiangong yuanshuai 田公元帅, and Lingya jiangjun 灵牙将军. ']
[' Yangzhici 仰止祠, Yongqingshê 永庆社 was built during the late Qing Dynasty, and was renovated both during the Republican period and after the founding of the PRC. The main gods are Zunzhu mingwang 尊主明王, Houtu furen 后土夫人, and Xu daren 徐大人. Secondary gods include Sanyi jiaozhu 三一教主, Zhanggong shengjun 张公圣君, Guansheng dijun 关圣帝君, and Puhua tianzun 普化天尊. ']
[' Zhaolinggong 昭灵宫, Zhongjingshê 中井社 was rebuilt in the Xuande period (1909–1911). 

completed parsing ./input/401-500.txt
[' 1) Dongdunjing Xianying xishê 东墩境显应西社 contains the Zhongxiaotan 忠孝坛, and was restored in 1989. The main god is Yanggong taishi 杨公太师. To the left is Tianqian shengmu 天前圣母, Lubu dashen 鲁部大神, and Lufu yima 鲁府姨妈, To the right is Yanggong taishi 杨公太师, Qitian dasheng 齐天大圣, Baifu daren 白府大人, and Wenwu zhushen 文武诸神. Further left are Wengong yuanshuai 温公元帅, and further right are Yaogong yuanshuai 姚公元帅, Zhangshi furen 张氏夫人, and Sanwei zhongjun 三位中军', ' Zhongxiaotan 忠孝坛 is linked to the Yunshuidian 云水殿 and is dedicated to a Nuo exorcistic god 傩神, and to Shi jiangjun 施将军 and Qitian dasheng 齐天大圣', ' Mingqingtang 明清堂 is dedicated to Sanyi jiaozhu 三一教主. The SJCDCB, 1992:10 record thirty-six initiates. ']
[' 1) Rendeli Puqianjing Yunfengtang 仁德里铺前境云 峰堂 was built in Qing Xuantong 3 (1911). In 1982 new god statues were carved and in 1990 the temple was restored. In front of the temple is a large canal, which is said to contain the lair of a Carp spirit. The main 

[' 1) Yulinjing Guoanshê 玉林境国安社 is linked with the Yulingong 玉林宫. The temple was built in Qing Qianlong period (1736–1795). The main gods are Zunzhu mingwang 尊主明王and Houtu furen 后土夫人. Secondary gods include Xuantian shangdi 玄天上帝, Jialuo dasheng 伽罗大圣, Liugong yuanshuai 刘公元帅, Zhanggong shengjun 张公圣君, Tiangong yuanshuai 田公元帅, Dutian yuanshuai 都天元帅, Qitian dasheng 齐天大圣, Taiyi xiangu 太乙仙姑, Wenchang dijun 文昌帝君, Jin jiangjun 金将军, and Wu jiangjun 吴将军', ' Tianhougong 天后宫 was built in the Republican period, and is dedicated to Tianshang shengmu 天上圣母. Secondary gods include Liugong shenghou 柳公圣侯 and Jingong shengjou 金公圣侯 and Yulin shengmu 毓麟圣母. ']
[' 1) Nipojing Gutang shexue 霓坡境古塘社学 is dedicated to Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Secondary gods include Zhanggong shengjun 张公圣君, Tiangong yuanshuai 田公元帅, Liugong yuanshuai 刘公元帅, Taiyi xiangu 太乙仙姑, Li sanfuren 李三夫人, Qitian dasheng 齐天大圣, Ciji zhenjun 慈济真君, Jixiang dajiang 吉祥大将, Lufu dashen 鲁府大神, Lufu yima 鲁府姨妈, and Fude zhengshen 福德正神. ']
[

completed parsing ./input/jiuliyang.txt
Completed


In [59]:
# Create csvs

import json, csv

with open('./output/village_master.json') as infile:
    
    indata = json.load(infile)
    
    # General data including 1) UID, 2) Name, 3) Name_zh, 4) Alliance, 5) Alliance_UID, 6) Alliance_zh, 7) Irrigation_Region
    keysToExtract = ["UID", "Name", "Name_zh", "Alliance", "Alliance_UID", "Alliance_zh", "Irrigation_Region"]
    genDataBuffer = []
        
    for entry in indata:
        
        outEntry = {}
        
        for var in entry:
            if var in keysToExtract:
                outEntry[var] = entry[var]
                
        genDataBuffer.append(outEntry)
        
    with open('./output/summary.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keysToExtract, delimiter=',')
        dict_writer.writeheader()
        dict_writer.writerows(genDataBuffer)
        
    # Raw_Text
    
    

In [84]:
# Data join

import json, csv

with open('./input/Village_Points.geojson') as geoJSONfile:
    
    # Load GeoJSON
    indata = json.load(geoJSONfile)
    
    # Load summary
    
    with open("./output/summary.csv") as csv_file:
        
        csv_buffer = []
        
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count != 0:
                line_count += 1
                
                csv_buffer.append({
                    "UID": row[0],
                    "Name": row[1],
                    "Name_zh": row[2],
                    "Alliance": row[3],
                    "Alliance_UID": row[4],
                    "Alliance_zh": row[5],
                    "Irrigation_Region": row[6]
                    })
            else:
                line_count += 1
                      
        print(f'Processed {line_count} lines.')
        
        for line in indata["features"]:
            
            # Merge files
            if line["properties"]["name"] == None:
                line["properties"]["name"] == line["properties"]['village_id_information_pingying']
                line["properties"]["township"] == line["properties"]['village_id_information_county']
                
            # find matching files
            matches = list(filter(lambda k: k["Name_zh"] == line["properties"]["hanzi"], csv_buffer))
            if len(matches) > 1:
                print(matches)

Processed 724 lines.
[{'UID': 'V016', 'Name': 'Houdun', 'Name_zh': '后墩', 'Alliance': 'Goukou qijing (Goukou sevenfold ritual alliance)', 'Alliance_UID': 'A003', 'Alliance_zh': '沟口七境', 'Irrigation_Region': 'Southern'}, {'UID': 'V308', 'Name': 'Houdun', 'Name_zh': '后墩', 'Alliance': 'Xiahuang qijing (Xiahuang sevenfold ritual alliance)', 'Alliance_UID': 'A065', 'Alliance_zh': '下黄七境', 'Irrigation_Region': 'Northern'}, {'UID': 'V397', 'Name': 'Houdun', 'Name_zh': '后墩', 'Alliance': 'Changfeng qijing (Changfeng sevenfold ritual alliance)', 'Alliance_UID': 'A086', 'Alliance_zh': '长丰七境', 'Irrigation_Region': 'Northern'}, {'UID': 'V401', 'Name': 'Houdun', 'Name_zh': '后墩', 'Alliance': 'Chenqiao sicun (Chenqiao four village ritual alliance)', 'Alliance_UID': 'A087', 'Alliance_zh': '陈桥四村', 'Irrigation_Region': 'Northern'}, {'UID': 'V431', 'Name': 'Houdun', 'Name_zh': '后墩', 'Alliance': 'Houzhou qijing (Houzhou sevenfold ritual alliance)', 'Alliance_UID': 'A099', 'Alliance_zh': '后卓七境', 'Irrigation_Re

In [2]:
# If you are not inside a QGIS console you first need to import
# qgis and PyQt classes you will use in this script as shown below:
import qgis.core 
# Get the project instance
QgsApplication.setPrefixPath("/usr/lib/qgis/", True)


ImportError: libspatialindex.so.5: cannot open shared object file: No such file or directory